this is looking at whether ppar spots are located around the periphery of blobs

In [1]:
import os
import numpy as np
import pandas as pd
%matplotlib qt
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import io
from skimage import measure, morphology
from scipy.ndimage.filters import gaussian_laplace
from skimage.filters import gaussian
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, binary_erosion, disk
from skimage.filters import threshold_otsu
from skimage.transform import rotate
from scipy import ndimage as ndi
from skimage.exposure import rescale_intensity
from skimage.feature import blob_log


# directories for images etc.

In [2]:
segm_img_dir = r'Z:\TAD\200428_adipogenesis_timecourses_revisited\190718_3t3l1_time_course_r5\filtered_segm_imgs' #directory of segmented images
cebp_segm_dir = r'Z:\TAD\200428_adipogenesis_timecourses_revisited\190718_3t3l1_time_course_r5\cebp_segm_imgs2'
dapi_segm_dir = r'Z:\TAD\200428_adipogenesis_timecourses_revisited\190718_3t3l1_time_course_r5\dapi_segm_imgs2'
orig_img_dir_base = r'Z:\TAD\190718_3t3l1_time_course_r5' #directory that has subdirectories with original images 

# Main script:
## determine the minimum distance of ppar puncta from cebpb hubs

In [10]:
'''this is just the global analysis to try and make sure things are generally working the way that I would expect'''

'''series of lists to be populated with values in the script'''

df_lst = []
point_distances_dapi =[]
point_distances_cebp = []
point_distances_rndm = []


os.chdir(cebp_segm_dir)

'''pull all of the .tif files out of the directory and put them into a list'''
cebp_segm_img_lst = []
for filename in os.listdir():
    if '.tif' in filename:
        cebp_segm_img_lst.append(filename)

        
filename_base = 'this is for a little later in code to check if files are already read into memory'

#########################################################
cebp_segm_img_lst = cebp_segm_img_lst[12:20] #this is temporary while working on code
################################################

time_count = 1
index_count = 1
for segm_cebp_img_filename in cebp_segm_img_lst:
    #print(str(datetime.now()) + '---working on ' + str(time_count) + ' of ' + str(len(cebp_segm_img_lst)))
    time_count += 1
    
    global_dict_temp = {}
    os.chdir(cebp_segm_dir)
    #global_dict_temp.update({'index_count' : index_count})
    index_count += 1
    #global_dict_temp.update({'segm_img_name' : segm_cebp_img_filename})
    

   
    
    cebp_segm_img = io.imread(segm_cebp_img_filename) #read in cebp single cell segmented image
    #cebp_segm_img = binary_erosion(cebp_segm_img, disk(3))
    
    '''some images have more than 10 images which throws off the naming conventions. This should handle this'''
    if len(segm_cebp_img_filename) == 32:
        segm_dapi_img_filename = segm_cebp_img_filename[:-19] +'dapi'+segm_cebp_img_filename[-15:]
        file_name_test = segm_cebp_img_filename[:-20]
        cell_number_ref = segm_cebp_img_filename[-5]
    if len(segm_cebp_img_filename) == 33:
        segm_dapi_img_filename = segm_cebp_img_filename[:-20] +'dapi'+segm_cebp_img_filename[-16:]
        file_name_test = segm_cebp_img_filename[:-21]
        cell_number_ref = segm_cebp_img_filename[-6:-4]
    
    os.chdir(dapi_segm_dir) #switch over to segmented dapi directory and read in segmented dapi image'''
    dapi_segm_img = io.imread(segm_dapi_img_filename) #read in the segmented dapi image
    
    
    '''the segmented nuclei image and the fluorescent images may already be in memory, in which case can save time by just leaving them
    there and not reading them in over and over'''
    if filename_base != file_name_test:
        '''read in segmented nuclei and fluorescent images'''
        filename_base = file_name_test #change filename_base to appropriate base name
        os.chdir(segm_img_dir) #switch over to the segmented nuceli image directory and pull out the individual nuclei'''
        nuclei_segm_img = io.imread(filename_base+'segm_parsed.tiff') #pull out the base filename in order to identify other images associated with this mask'''
        os.chdir(orig_img_dir_base+'\\'+filename_base[:-4]) #switch over to the directory with the original fluoresence images and read in dapi, cebp and ppar images'''
        dapi_orig_img = io.imread(filename_base+'c1.tif') #read in original dapi image
        ppar_orig_img = io.imread(filename_base+'c3.tif') #read in original ppar image
    
        cebp_orig_img = io.imread(filename_base+'c4.tif') #read in original cebp image
    ##############################################################################################################################
    ############################ whole cell analysis######################################################################
    #######################################################################################################################
    '''pull out the individual nuclei from this image that corresponds to the segmented cebp and dapi images'''
    labeled_nuclei_segm_img = measure.label(nuclei_segm_img) #label the objects within the image 
    nuclei_number = int(cell_number_ref) #find the associated nucleus in the image based on the filename
    nucleus_segm_img = np.array(labeled_nuclei_segm_img == nuclei_number).astype('uint16') #pull out the single nuclues
    

   
    
    #cebp_segm_img = io.imread(segm_cebp_img_filename) #read in cebp single cell segmented image
    #cebp_segm_img = binary_erosion(cebp_segm_img, disk(3))
    
    '''some images have more than 10 images which throws off the naming conventions. This should handle this'''
    if len(segm_cebp_img_filename) == 32:
        segm_dapi_img_filename = segm_cebp_img_filename[:-19] +'dapi'+segm_cebp_img_filename[-15:]
        file_name_test = segm_cebp_img_filename[:-20]
        cell_number_ref = segm_cebp_img_filename[-5]
    if len(segm_cebp_img_filename) == 33:
        segm_dapi_img_filename = segm_cebp_img_filename[:-20] +'dapi'+segm_cebp_img_filename[-16:]
        file_name_test = segm_cebp_img_filename[:-21]
        cell_number_ref = segm_cebp_img_filename[-6:-4]
    
    os.chdir(dapi_segm_dir) #switch over to segmented dapi directory and read in segmented dapi image'''
    dapi_segm_img = io.imread(segm_dapi_img_filename) #read in the segmented dapi image
    
    
    '''the segmented nuclei image and the fluorescent images may already be in memory, in which case can save time by just leaving them
    there and not reading them in over and over'''
    if filename_base != file_name_test:
        '''read in segmented nuclei and fluorescent images'''
        filename_base = file_name_test #change filename_base to appropriate base name
        os.chdir(segm_img_dir) #switch over to the segmented nuceli image directory and pull out the individual nuclei'''
        nuclei_segm_img = io.imread(filename_base+'segm_parsed.tiff') #pull out the base filename in order to identify other images associated with this mask'''
        os.chdir(orig_img_dir_base+'\\'+filename_base[:-4]) #switch over to the directory with the original fluoresence images and read in dapi, cebp and ppar images'''
        dapi_orig_img = io.imread(filename_base+'c1.tif') #read in original dapi image
        ppar_orig_img = io.imread(filename_base+'c3.tif') #read in original ppar image
        cebp_orig_img = io.imread(filename_base+'c4.tif') #read in original cebp image
        
    ##############################################################################################################################
    ############################ whole cell analysis######################################################################
    #######################################################################################################################
    '''pull out the individual nuclei from this image that corresponds to the segmented cebp and dapi images'''
    labeled_nuclei_segm_img = measure.label(nuclei_segm_img) #label the objects within the image 
    nuclei_number = int(cell_number_ref) #find the associated nucleus in the image based on the filename
    nucleus_segm_img = np.array(labeled_nuclei_segm_img == nuclei_number).astype('uint16') #pull out the single nuclues
    
    
    
    
    '''this iterates ove each of the blobs rotates and translates them to somewhere random in the nuclues'''
    
    nucleus_erode = binary_erosion(nucleus_segm_img, disk(15)) #reode the segmented nucleus image so won't chooose points too close to the edge a little later on
    
    labeled_blob_segm_img = measure.label(cebp_segm_img) #label the blobs within the image 
    
    new_blob_img = np.zeros([nucleus_segm_img.shape[0], nucleus_segm_img.shape[1]]).astype('uint16') #set up array of zeros with same shape as original image
    
    for labeled_blob in np.unique(labeled_blob_segm_img)[1:]: #iterate over the blobs
        indy_blob = np.array(labeled_blob_segm_img == labeled_blob).astype('uint8') #choose an individual blob to look at
        minr_b, minc_b, maxr_b, maxc_b = measure.regionprops(indy_blob)[0].bbox # find coordinates of a bounding box around the individual blob
        angle = int(np.random.randint(0, 360, 1)) #choose a random angle to ratate the blob
        indy_blob_rotate = rotate(indy_blob[minr_b:maxr_b, minc_b: maxc_b], angle) # rotate the blob
        thresh = threshold_otsu(indy_blob_rotate)
        indy_blob_rotate = np.array(indy_blob_rotate > thresh).astype('uint8') # turn rotated blob back into binary ineteger
        y_center_blob, x_center_blob = measure.regionprops(indy_blob_rotate)[0].centroid # find the center of the rotated blob
        x_center_blob = int(x_center_blob)
        y_center_blob = int(y_center_blob)
        nucleus_coordinate = np.argwhere(nucleus_erode == 1) # choose random new coordinates within the nucleus
        rand_coord_numb = int(np.random.randint(0, len(nucleus_coordinate), 1))
        new_y_center, new_x_center = nucleus_coordinate[rand_coord_numb] # record the x and y coordinates of the randomly chosen new center
        translate_y = int(new_y_center) - y_center_blob #amount to translate rotated blob
        translate_x = int(new_x_center) - x_center_blob #amount to translate rotated blob
        orig_rotated_blob_coords = np.argwhere(indy_blob_rotate == 1) # generated list of original coordinates for the rotated blob
        ycoords_rotated_orig_pos = orig_rotated_blob_coords[:, 0] #coordinates for the 
        xcoords_rotated_orig_pos = orig_rotated_blob_coords[:, 1]
        ycoords_rotated_new_pos = ycoords_rotated_orig_pos + translate_y
        xcoords_rotated_new_pos = xcoords_rotated_orig_pos + translate_x
        
        for new_y, new_x in zip(ycoords_rotated_new_pos, xcoords_rotated_new_pos):
            new_blob_img[new_y, new_x] = 1
        new_blob_img = np.multiply(nucleus_segm_img, new_blob_img)

#nuclei_number = int(cell_number_ref) #find the associated nucleus in the image based on the filename
#nucleus_segm_img = np.array(labeled_nuclei_segm_img == nuclei_number).astype('uint16') #pull out the single nuclues
    
    
    
    minr, minc, maxr, maxc = measure.regionprops(nucleus_segm_img)[0].bbox
    ppar_normed_img = ppar_orig_img / np.quantile(ppar_orig_img[minr:maxr, minc:maxc], .98)
    ppar_dots = blob_log(ppar_normed_img[minr:maxr, minc:maxc], max_sigma=5, num_sigma=3, threshold=.2)
    #ppar_dots = blob_log(ppar_nucleus_img, max_sigma=3, num_sigma=1, threshold=.05)
    ppar_nucleus_img = np.multiply(ppar_normed_img, nucleus_segm_img)
    
    blob_erosion = morphology.binary_erosion(cebp_segm_img, morphology.disk(1))
    cebp_ring = np.subtract(cebp_segm_img, blob_erosion)
    
    ringy = np.argwhere(cebp_ring == 1)[:, 0]
    ringx = np.argwhere(cebp_ring == 1)[:, 1]
    
    blob_x_cebp = np.argwhere(cebp_segm_img == 1)[:, 1]
    blob_y_cebp = np.argwhere(cebp_segm_img == 1)[:, 0]
    
    blob_x_dapi = np.argwhere(dapi_segm_img == 1)[:, 1]
    blob_y_dapi = np.argwhere(dapi_segm_img == 1)[:, 0]
    
    blob_x_rndm = np.argwhere(new_blob_img == 1)[:, 1]
    blob_y_rndm = np.argwhere(new_blob_img == 1)[:, 0]
    
    
    ppar_x = ppar_dots[:, 1]
    ppar_x = ppar_x + minc
    ppar_y = ppar_dots[:, 0]
    ppar_y = ppar_y + minr
    
    for x_p, y_p in zip(ppar_x, ppar_y):
        min_dist_cebp = 1000
        min_dist_dapi = 1000
        min_dist_rndm = 1000
        for x_c, y_c in zip(blob_x_cebp, blob_y_cebp):
            dist_ = np.sqrt((x_p - x_c) **2 + (y_p - y_c) ** 2)
            min_dist_cebp = np.min([min_dist_cebp, dist_])
        for x_d, y_d in zip(blob_x_dapi, blob_y_dapi):
            dist_ = np.sqrt((x_p - x_d) **2 + (y_p - y_d) ** 2)
            min_dist_dapi = np.min([min_dist_dapi, dist_])
        for x_r, y_r in zip(blob_x_rndm, blob_y_rndm):
            dist_ = np.sqrt((x_p - x_r)**2 + (y_p - y_r) ** 2)
            min_dist_rndm = np.min([min_dist_rndm, dist_])
        point_distances_cebp.append(min_dist_cebp)
        point_distances_dapi.append(min_dist_dapi)
        point_distances_rndm.append(min_dist_rndm)
        
    #ppar_blob = blob_log(ppar_nucleus_img, max_sigma=8, num_sigma=5, threshold=.05)   
    
   #y = ppar_dots[:, 0]
   #x = 
   #
   #y = y +minr
   #x = x +minc
   #
   # fig, ax = plt.subplots()
   # #ax = axes.ravel()
   # for idx in [0]:
   #     ax.imshow(ppar_nucleus_img)
   #     #ax.imshow(ppar_orig_img)
   #     ax.scatter(ringx, ringy, color = 'y', s = .15)
   #     for blob in ppar_dots:
   #         y, x, r = blob
   #         y = y + minr
   #         x = x + minc
   #         c = plt.Circle((x, y), r, color='r', linewidth=1, fill=False)
   #         ax.add_patch(c)
   #         ax.set_axis_off()
    


    
    
    
    

In [9]:
cebp_segm_img_lst

['stad3-42-xy3_cebp_segm_cell11.tif', 'stad3-42-xy3_cebp_segm_cell12.tif']

# Make graphs from lists

In [11]:
ax = plt.hist(np.array(point_distances_cebp)[np.array(point_distances_cebp) > 0], density = True, cumulative= 1, bins = np.linspace(1, 4, 40))
ax = plt.hist(np.array(point_distances_dapi)[np.array(point_distances_dapi) > 0], density = True, cumulative= 1, bins = np.linspace(1, 4, 40), color = 'r')

In [12]:
n_c, b_c, p_c = plt.hist(np.array(point_distances_cebp), density = True, cumulative = True, bins = np.linspace(0, 40, 41))
n_d, b_d, p_d =  plt.hist(np.array(point_distances_dapi), density = True, cumulative= True, bins = np.linspace(0, 40, 41))
n_r, b_r, p_r =  plt.hist(np.array(point_distances_rndm), density = True, cumulative= True, bins = np.linspace(0, 40, 41))

In [13]:
plt.style.use('dark_background')

In [14]:
plt.plot(b_c[:-1], n_c)
plt.plot(b_d[:-1], n_d)
plt.plot(b_d[:-1], n_r)

In [21]:
ax = plt.hist(np.array(point_distances_cebp), density = True, bins = np.linspace(0, 40, 40))
ax = plt.hist(np.array(point_distances_rndm), density = True, bins = np.linspace(0, 40, 40), color = 'r')

# Make dataframe from lists

## 1 = cebp, 2 = dapi, 3 = random

In [15]:
''' make a dataframe out of the lists'''
running_df_lst = []
lst_o_lsts = [point_distances_cebp, point_distances_dapi, point_distances_rndm]

count = 0

for i in lst_o_lsts:
    hubs_lst = np.zeros(len(i))
    hubs_lst += count
    df_ = pd.DataFrame({'hub' : hubs_lst,
                       'dist' : i})
    running_df_lst.append(df_)
    count+= 1
df_whole = pd.concat(running_df_lst)


# Make graphs from df 

In [16]:
from scipy import stats

In [17]:
stats.ttest_ind(point_distances_dapi, point_distances_rndm)

Ttest_indResult(statistic=-1.492929550120054, pvalue=0.1360965954381619)

In [18]:

df_whole['um'] = df_whole.dist * .065

In [19]:
ax = sns.stripplot(df_whole.hub, df_whole.um)
ax = sns.pointplot(df_whole.hub, df_whole.um , join = False, color = 'r', ci = 'sd')

In [20]:
ax = sns.violinplot(df_whole.hub, df_whole.um)

In [35]:
df_ = df_whole.loc[df_whole.dist <= np.quantile(df_whole.dist, .98)]
ax = sns.violinplot(df_.hub, df_.um)

In [29]:
point_distances = np.array(point_distances)
point_distances_dapi = np.array(point_distances_dapi)

point_distances_ = point_distances[point_distances > 0]
point_distances_dapi_ = point_distances_dapi[point_distances_dapi > 0]

stats.ttest_ind(point_distances_, point_distances_dapi_)

Ttest_indResult(statistic=-21.289723386696497, pvalue=8.764928998969672e-94)

In [21]:
os.chdir(r'Y:\TAD\200904_aggregate_puncta_analysis\ppar')

In [22]:
df_whole.to_csv('pppar_puncta_dists.csv')

In [30]:
binner = np.arange(0, 6, .1)

ax = sns.kdeplot(df_whole.loc[df_whole.hub == 0].um, label = 'CEBPb aggregate')
ax = sns.kdeplot(df_whole.loc[df_whole.hub == 1].um, label = 'heterochromatin')
ax = sns.kdeplot(df_whole.loc[df_whole.hub == 2].um, label = 'random mask')



In [32]:
df_ = df_whole.loc[df_whole.um < np.quantile(df_whole.um, .95)]

ax = sns.violinplot(df_.hub, df_.um)

# working on ripley's functions

In [16]:
np.max(df_whole.um) # maximum distance to take graph out to

4.4661196804384895

In [18]:
len(df_whole) / 3 # number of PPARg puncta

85.0

In [20]:
plt.hist(df_whole.um, density= True, cumulative= 1)

(array([0.27058824, 0.54901961, 0.72941176, 0.81960784, 0.88627451,
        0.92941176, 0.95686275, 0.98431373, 0.99607843, 1.        ]),
 array([0.        , 0.44661197, 0.89322394, 1.3398359 , 1.78644787,
        2.23305984, 2.67967181, 3.12628378, 3.57289574, 4.01950771,
        4.46611968]),
 <a list of 10 Patch objects>)

# Random stuff that I wrote along the way

In [14]:
fig, axes = plt.subplots(2,4)
axes = axes.flatten()

axes[0].imshow([minr:maxr, minc:maxc])
axes[1].imshow(nucleus_segm_img[minr:maxr, minc:maxc])

SyntaxError: invalid syntax (<ipython-input-14-13c61b48c6b7>, line 4)

In [55]:
'''this makes an image of the perimeter of nucleus mask. this is over kill as I could have just done a dilation and erosion then 
did argwhere'''
ppar_dots = blob_log(ppar_normed_img[minr:maxr, minc:maxc], max_sigma=5, num_sigma=3, threshold=.2)  
    
y = ppar_dots[:, 0]
x = ppar_dots[:, 1]

y = y +minr
x = x +minc

fig, ax = plt.subplots()
#ax = axes.ravel()
for idx in [0]:
    ax.imshow(ppar_nucleus_img)
    #ax.imshow(ppar_orig_img)
    ax.scatter(ringx, ringy, color = 'y', s = .5)
    for blob in ppar_dots:
        y, x, r = blob
        y = y + minr
        x = x + minc
        c = plt.Circle((x, y), 2, color='r', linewidth=1, fill=False)
        ax.add_patch(c)
        ax.set_axis_off()

In [57]:
fig, axes = plt.subplots()

axes.imshow(ppar_nucleus_img)

In [104]:
fig, axes = plt.subplots(2,2)
axes = axes.flatten()

axes[0].imshow(perim_img1[minr:maxr, minc:maxc])
axes[1].imshow(perim_img2[minr:maxr, minc:maxc])
axes[2].imshow(perim_img3[minr:maxr, minc:maxc])
axes[3].imshow(perim_img4[minr:maxr, minc:maxc])

In [67]:
fig, axes = plt.subplots(2,3)

axes = axes.flatten()


axes[1].imshow(cebp_orig_img[minr:maxr, minc:maxc])
axes[0].imshow(dapi_orig_img[minr:maxr, minc:maxc])
axes[2].imshow(ppar_orig_img[minr:maxr, minc:maxc])
axes[4].imshow(cebp_segm_img[minr:maxr, minc:maxc])
axes[3].imshow(dapi_segm_img[minr:maxr, minc:maxc])
axes[5].imshow(new_blob_img[minr:maxr, minc:maxc])
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);

In [47]:
ppar_blob = blob_log(ppar_nucleus_img, max_sigma=8, num_sigma=5, threshold=.05)   

y = ppar_dots[:, 0]
x = ppar_dots[:, 1]

y = y +minr
x = x +minc
#
fig, ax = plt.subplots()
 #ax = axes.ravel()
for idx in [0]:
    ax.imshow(ppar_nucleus_img)
     #ax.imshow(ppar_orig_img)
    ax.scatter(ringx, ringy, color = 'y', s = .15)
    for blob in ppar_dots:
        y, x, r = blob
        y = y + minr
        x = x + minc
        c = plt.Circle((x, y), 2, color='r', linewidth=2, fill=False)
        ax.add_patch(c)
        ax.set_axis_off()
 

In [120]:
from skimage.morphology import binary_erosion, disk

binary_img_erode = binary_erosion(binary_img, disk(1))
binary_img_perim = binary_img - binary_img_erode

np.argwhere(binary_img_perim == 1)

array([[1236, 1386],
       [1236, 1387],
       [1236, 1388],
       ...,
       [1453, 1324],
       [1454, 1311],
       [1454, 1312]], dtype=int64)

In [130]:
def one_or_zero(percint_one = .15, 1):
    rand_ = float(np.random.random_sample(1))
    if rand_ < percint_one:
        return(1)
    else:
        return(0)

In [135]:
rand_img = np.zeros([nucleus_segm_img.shape[0], nucleus_segm_img.shape[1]]).astype('uint16')

portion_blob = np.sum(cebp_segm_img) / np.sum(nucleus_segm_img)

for yer in range(minr -1,maxr +1):
    for xer in range(minc - 1, maxc  + 1):
        rand_img[yer, xer] = one_or_zero(portion_blob)
rand_img = np.multiply(rand_img, nucleus_segm_img)

In [158]:
prob1_lst = []
prob0_lst = []

for yer, xer in np.argwhere(nucleus_segm_img == 1):
    ybefore = cebp_segm_img[yer -1, xer]
    yafter = cebp_segm_img[yer +1, xer]
    xbefore =cebp_segm_img[yer, xer -1]
    xafter = cebp_segm_img[yer, xer + 1]
    mean_around = (ybefore+yafter+xbefore+xafter) / 4
    if cebp_segm_img[yer, xer] ==1:
        prob1_lst.append(mean_around)
    else:
        prob0_lst.append(mean_around)

            
prob1_blob = np.sum(prob1_lst) / len(prob1_lst)
prob0_blob = np.sum(prob0_lst) / len(prob0_lst)


rndm_blob_img = np.copy(cebp_segm_img)

for index in range(100):
    for yer, xer in np.argwhere(nucleus_segm_img == 1):
        new_x = xer.copy()
        new_y = yer.copy()
        rand = float(np.random.random_sample(1))
        if rand <= .25:
            new_x += 1
        if rand > .25 and rand <= .5:
            new_x -= 1
        if rand > .5 and rand <= .75:
            new_y += 1
        else:
            new_y -= 1   
        if rndm_blob_img[yer,xer] == 1:
            rndm_blob_img[new_y, new_x] = one_or_zero(prob1_blob)
        else:
            rndm_blob_img[new_y, new_x] = one_or_zero(prob0_blob)
        


In [156]:
prob0_blob

0.012246405637324544

In [159]:
fig, axes = plt.subplots(1, 3)
axes = axes.flatten()

axes[0].imshow(nucleus_segm_img[minr:maxr, minc:maxc])
axes[1].imshow(cebp_segm_img[minr:maxr, minc:maxc])
axes[2].imshow(rndm_blob_img[minr:maxr, minc:maxc])

In [14]:
'''this iterates ove each of the blobs rotates and translates them to somewhere random in the nuclues'''

from skimage.transform import rotate

nucleus_erode = binary_erosion(nucleus_segm_img, disk(15)) #reode the segmented nucleus image so won't chooose points too close to the edge a little later on

labeled_blob_segm_img = measure.label(cebp_segm_img) #label the blobs within the image 

new_blob_img = np.zeros([nucleus_segm_img.shape[0], nucleus_segm_img.shape[1]]).astype('uint16') #set up array of zeros with same shape as original image

for labeled_blob in np.unique(labeled_blob_segm_img)[1:]: #iterate over the blobs
    indy_blob = np.array(labeled_blob_segm_img == labeled_blob).astype('uint8') #choose an individual blob to look at
    minr_b, minc_b, maxr_b, maxc_b = measure.regionprops(indy_blob)[0].bbox # find coordinates of a bounding box around the individual blob
    angle = int(np.random.randint(0, 360, 1)) #choose a random angle to ratate the blob
    indy_blob_rotate = rotate(indy_blob[minr_b:maxr_b, minc_b: maxc_b], angle) # rotate the blob
    thresh = threshold_otsu(indy_blob_rotate)
    indy_blob_rotate = np.array(indy_blob_rotate > thresh).astype('uint8') # turn rotated blob back into binary ineteger
    y_center_blob, x_center_blob = measure.regionprops(indy_blob_rotate)[0].centroid # find the center of the rotated blob
    x_center_blob = int(x_center_blob)
    y_center_blob = int(y_center_blob)
    nucleus_coordinate = np.argwhere(nucleus_erode == 1) # choose random new coordinates within the nucleus
    rand_coord_numb = int(np.random.randint(0, len(nucleus_coordinate), 1))
    new_y_center, new_x_center = nucleus_coordinate[rand_coord_numb] # record the x and y coordinates of the randomly chosen new center
    translate_y = int(new_y_center) - y_center_blob #amount to translate rotated blob
    translate_x = int(new_x_center) - x_center_blob #amount to translate rotated blob
    orig_rotated_blob_coords = np.argwhere(indy_blob_rotate == 1) # generated list of original coordinates for the rotated blob
    ycoords_rotated_orig_pos = orig_rotated_blob_coords[:, 0] #coordinates for the 
    xcoords_rotated_orig_pos = orig_rotated_blob_coords[:, 1]
    ycoords_rotated_new_pos = ycoords_rotated_orig_pos + translate_y
    xcoords_rotated_new_pos = xcoords_rotated_orig_pos + translate_x
    
    for new_y, new_x in zip(ycoords_rotated_new_pos, xcoords_rotated_new_pos):
        new_blob_img[new_y, new_x] = 1
    new_blob_img = np.multiply(nucleus_segm_img, new_blob_img)

#nuclei_number = int(cell_number_ref) #find the associated nucleus in the image based on the filename
#nucleus_segm_img = np.array(labeled_nuclei_segm_img == nuclei_number).astype('uint16') #pull out the single nuclues

IndexError: index 2044 is out of bounds for axis 1 with size 2044

In [15]:
fig, axes = plt.subplots(1,3)
axes = axes.flatten()

axes[0].imshow(nucleus_segm_img[minr:maxr, minc:maxc])
axes[1].imshow(cebp_segm_img[minr:maxr, minc:maxc])
axes[2].imshow(new_blob_img[minr:maxr, minc:maxc])

In [227]:
fig, axes = plt.subplots(1,2)
axes = axes.flatten()

axes[0].imshow(nucleus_segm_img[minr:maxr, minc:maxc])
axes[1].imshow(nucleus_erode[minr:maxr, minc:maxc])
#plt.imshow(indy_blob_rotate)


In [188]:

jnk = rotate(indy_blob[minr_b:maxr_b, minc_b: maxc_b], angle)
thresh = threshold_otsu(jnk)
jnk = jnk > thresh

In [189]:
fig, axes = plt.subplots(1,2)
axes = axes.flatten()

axes[0].imshow(indy_blob[minr_b: maxr_b, minc_b: maxc_b])
axes[1].imshow(jnk)

In [176]:
np.unique(jnk)

array([0.00000000e+00, 2.97958641e-05, 6.71230996e-05, 7.55971266e-05,
       1.95598333e-04, 2.23986589e-04, 2.23986589e-04, 3.43163560e-04,
       3.43163560e-04, 3.44540842e-04, 4.71890364e-04, 4.81401613e-04,
       5.97406956e-04, 6.18262385e-04, 6.25429126e-04, 6.58429777e-04,
       7.31299742e-04, 8.06422993e-04, 8.44616341e-04, 9.01600491e-04,
       3.06883858e-03, 3.06883858e-03, 3.20561144e-03, 3.22832481e-03,
       3.22832481e-03, 3.31286190e-03, 3.32229308e-03, 3.37826121e-03,
       3.44964822e-03, 3.44972267e-03, 3.45211158e-03, 3.51218077e-03,
       3.58897235e-03, 3.60113981e-03, 3.65412910e-03, 3.66337502e-03,
       3.72105530e-03, 3.72583312e-03, 3.79534939e-03, 3.80023579e-03,
       3.85791607e-03, 3.89750646e-03, 3.91513576e-03, 3.92156863e-03])

In [73]:
cebp_sing_cell = np.multiply(cebp_segm_img, nucleus_segm_img)

np.sum(cebp_sing_cell) / np.sum(nucleus_segm_img)

0.09849125233740619

In [74]:
np.sum(new_blob_img) / np.sum(nucleus_segm_img)

0.07758907758907758

In [75]:
new_blob_img_erode = binary_erosion(new_blob_img, disk(1))

new_blob_ring = new_blob_img - new_blob_img_erode

cebp_sing_cell_erode = binary_erosion(cebp_sing_cell, disk(1))

cebp_sing_cell_ring = cebp_sing_cell - cebp_sing_cell_erode

In [76]:
np.sum(cebp_sing_cell_ring)

1160

In [77]:
np.sum(new_blob_ring)

892

In [138]:
np.max(measure.label(cebp_segm_img)) 

23

In [37]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d
points = np.argwhere(nucleus_segm_img == 1)
hull = ConvexHull(points)

In [44]:
for simplex in hull.simplices:
    plt.scatter(points[simplex, 0], points[simplex, 1])

In [43]:
hull.simplices

array([[   17,     0],
       [  308,   231],
       [ 9863, 13598],
       [16612, 23197],
       [25301, 23197],
       [ 2504,   566],
       [ 8409,  5857],
       [   54,    17],
       [  865,  1429],
       [  865,   308],
       [ 7205,  6045],
       [ 7205,  9863],
       [ 2214,  1429],
       [ 2214,  2503],
       [ 4592,  6045],
       [39036, 39014],
       [38980, 38775],
       [38980, 39014],
       [32773, 30999],
       [35549, 34405],
       [15211, 13598],
       [15211, 16612],
       [27349, 30999],
       [27349, 25301],
       [   18,     0],
       [   18,    55],
       [  162,   566],
       [  162,    55],
       [ 2802,  5857],
       [ 2802,  2504],
       [20841, 22490],
       [20841, 18020],
       [15444, 13599],
       [15444, 18020],
       [10935, 13599],
       [39015, 39037],
       [39015, 38981],
       [38938, 38889],
       [38938, 38981],
       [37445, 36830],
       [37445, 37670],
       [38498, 38889],
       [38498, 37670],
       [243

In [4]:
'''this makes an image of the perimeter of nucleus mask. this is over kill as I could have just done a dilation and erosion then 
did argwhere'''
ppar_dots = blob_log(ppar_normed_img[minr:maxr, minc:maxc], max_sigma=5, num_sigma=3, threshold=.2)  
    
y = ppar_dots[:, 0]
x = ppar_dots[:, 1]

y = y +minr
x = x +minc

fig, ax = plt.subplots()
#ax = axes.ravel()
for idx in [0]:
    ax.imshow(ppar_nucleus_img)
    #ax.imshow(ppar_orig_img)
    ax.scatter(ringx, ringy, color = 'y', s = .5)
    for blob in ppar_dots:
        y, x, r = blob
        y = y + minr
        x = x + minc
        c = plt.Circle((x, y), 2, color='r', linewidth=1, fill=False)
        ax.add_patch(c)
        ax.set_axis_off()

In [5]:
ppar_testing = ppar_orig_img[minr:maxr, minc:maxc]

In [32]:


def ab_dot_detecter(dotty_img, log_sigma_max = 20, log_sigma_num = 5, log_thresh = .01, norm_p = 1):
    normed_img = dotty_img /  np.quantile(dotty_img, norm_p)
    dots = blob_log(normed_img, max_sigma= log_sigma_max, num_sigma= log_sigma_num, threshold= log_thresh)
    
    dots_y_coord = dots[:, 0]
    dots_x_coord = dots[:, 1]
    
    return(dots_y_coord, dots_x_coord)

In [33]:
y_, x_ = ab_dot_detecter(ppar_testing)

C:\Users\LegantLab\anaconda3\lib\site-packages\skimage\feature\blob.py:125: RuntimeWarning: invalid value encountered in double_scalars
  r1 = blob1[-1] / blob2[-1]
C:\Users\LegantLab\anaconda3\lib\site-packages\skimage\feature\blob.py:126: RuntimeWarning: divide by zero encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)
C:\Users\LegantLab\anaconda3\lib\site-packages\skimage\feature\blob.py:127: RuntimeWarning: divide by zero encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)
C:\Users\LegantLab\anaconda3\lib\site-packages\skimage\feature\blob.py:129: RuntimeWarning: invalid value encountered in subtract
  d = np.sqrt(np.sum((pos2 - pos1)**2))


In [ ]:
fig, ax = plt.subplots()
#ax = axes.ravel()
for idx in [0]:
    ax.imshow(ppar_nucleus_img)
    #ax.imshow(ppar_orig_img)
    ax.scatter(ringx, ringy, color = 'y', s = .5)
    for blob in ppar_dots:
        y, x, r = blob
        y = y + minr
        x = x + minc
        c = plt.Circle((x, y), 2, color='r', linewidth=1, fill=False)
        ax.add_patch(c)
        ax.set_axis_off()

In [35]:
fig, ax = plt.subplots()

ax.imshow(ppar_nucleus_img)
c = plt.Circle((x_, y_), 2, color='r', linewidth=1, fill=False)
ax.add_patch(c)

ValueError: setting an array element with a sequence.

In [37]:
plt.add_patch(c)

AttributeError: module 'matplotlib.pyplot' has no attribute 'add_patch'

In [17]:
normed_img_ = dotty_img_ /  np.quantile(dotty_img_, 1)
dots_ = blob_log(normed_img_, max_sigma = 5, num_sigma= 3, threshold= .2)

In [39]:
plt.imshow(ppar_testing)
plt.scatter(x_, y_, s=80, facecolors='none', edgecolors='r')